In [1]:
import numpy as np # linear algebra
import pandas as pd
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/smile-twitter-emotion-dataset/smile-annotations-final.csv
/kaggle/input/smile-twitter-emotion-dataset/smile-annotations-final.json


In [2]:
df = pd.read_csv(
    '../input/smile-twitter-emotion-dataset/smile-annotations-final.csv',
    names=['id', 'text', 'category']
)
df.set_index('id', inplace=True)

In [3]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [4]:
df = df[~df.category.str.contains('\|')]
df = df[df.category != 'nocode']

In [5]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [6]:
classes = df.category.unique()
class_labels = {}
for index, cls in enumerate(classes):
    class_labels[cls] = index
df['label'] = df.category.replace(class_labels)

In [7]:
X_train, X_val, Y_test, Y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    stratify=df.label.values
)

In [8]:
df['data_type'] = ['not_set'] * df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [9]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

In [10]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [11]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [12]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [13]:
dataset_train = TensorDataset(input_ids_train,
                             attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val,
                           attention_masks_val, labels_val)

In [14]:
from transformers import BertForSequenceClassification

In [15]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(classes),
    output_attentions=False,
    output_hidden_states=False
)

In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [17]:
batch_size = 32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=SequentialSampler(dataset_val),
    batch_size=batch_size
)

In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [19]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

In [20]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train) * epochs
)

In [21]:
from sklearn.metrics import f1_score

In [31]:
def f1_score_func(preds, labels):
    preds = np.argmax(preds, axis=1).flatten()
    labels = labels.flatten()
    return f1_score(labels, preds, average='weighted')

def accuracy(preds, labels):
    labels_class = {v: k for k, v in class_labels.items()}
    labels = labels.flatten()
    preds = np.argmax(preds, axis=1).flatten()
    for label in np.unique(labels):
        y_preds = preds[labels == label]
        y_true  = labels[labels == label]
        print(f'Class {labels_class[label]}')
        print(f'Accuracy {len(y_preds[y_preds == label])} / {len(y_true)}')

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids'     : batch[0],
            'attention_mask': batch[1],
            'labels'        : batch[2]
        }
        
        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
    loss_val_avg = loss_val_total / len(dataloader_val)
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    
    return loss_val_avg, predictions, true_vals

In [26]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, 
                       desc='Epoch {:1d}'.format(epoch),
                       leave=False,
                       disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids'     : batch[0],
            'attention_mask': batch[1],
            'labels'        : batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})
        
    #torch.save(model.state_dict(), f'Model/BE_ft_epoch{epoch}.model')
    tqdm.write(f'\nEpoch{epoch}')
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training_loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation_loss: {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')


Epoch1
Training_loss: 0.738680723309517
Validation_loss: 0.6858798818928855
F1 score (weighted): 0.7152814888821766



Epoch2
Training_loss: 0.5860011264681816
Validation_loss: 0.6075828501156398
F1 score (weighted): 0.7639546175365834



Epoch3
Training_loss: 0.49272343292832377
Validation_loss: 0.541605532169342
F1 score (weighted): 0.7955360313827662



Epoch4
Training_loss: 0.42024144250899553
Validation_loss: 0.5205472665173667
F1 score (weighted): 0.8148260748397559



Epoch5
Training_loss: 0.3683679237961769
Validation_loss: 0.5132364588124412
F1 score (weighted): 0.8076128513348244



Epoch6
Training_loss: 0.3168510891497135
Validation_loss: 0.5358040375368935
F1 score (weighted): 0.812073128665057



Epoch7
Training_loss: 0.2978240855038166
Validation_loss: 0.5025289527007512
F1 score (weighted): 0.8470896915739965



Epoch8
Training_loss: 0.2743216924369335
Validation_loss: 0.49852611337389263
F1 score (weighted): 0.8517016915949456



Epoch9
Training_loss: 0.25890675764530896
Validation_loss: 0.5015531608036586
F1 score (weighted): 0.8555389177609838



Epoch10
Training_loss: 0.2567145091481507
Validation_loss: 0.5015531608036586
F1 score (weighted): 0.8555389177609838



In [27]:
_, predictions, true_vals = evaluate(dataloader_val)

In [32]:
accuracy(predictions, true_vals)

Class happy
Accuracy 166 / 171
Class not-relevant
Accuracy 25 / 32
Class angry
Accuracy 5 / 9
Class disgust
Accuracy 0 / 1
Class sad
Accuracy 0 / 5
Class surprise
Accuracy 0 / 5
